In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torchmetrics

import os
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from pytorch_lightning.loggers import TensorBoardLogger
from torchvision import datasets, transforms , models
from PIL import Image
import pytorch_lightning as pl
device = torch.device("cuda" if torch.cuda.is_available else "cpu")


In [3]:
train_folder = "Data/train"
test_folder = "Data/test1"

train_list = os.listdir(train_folder)
test_list = os.listdir(test_folder)

In [4]:
print("Number of training images: ", len(train_list))
print("Number of test images: ", len(test_list))

Number of training images:  25000
Number of test images:  12500


In [ ]:
imgpath = os.path.join(train_folder, train_list[0])

In [7]:
transformation = transforms.Compose([
    transforms.Resize((60,60)),
    transforms.ToTensor(),
])

In [21]:
class Dataset():
    def __init__(self, data_folder, data_list, transform = None):
        self.data_folder = data_folder
        self.data_list = data_list
        self.transform = transform

    def __len__(self):
            return int(len(self.data_list))
    
    def __getitem__(self, idx):
         imgpath = os.path.join(self.data_folder, self.data_list[idx])
         img = Image.open(imgpath)
         if 'cat' in imgpath:
             label = 0
         else:
             label = 1
         if self.transform is not None:
             img = self.transform(img)
         return(img, label)    
                


In [24]:
train = Dataset(train_folder, train_list, transformation)
test = Dataset(test_folder, test_list, transformation)

train, val = torch.utils.data.random_split(train, [20000, 5000])

train_loader = DataLoader(train, batch_size = 64, shuffle = True)
val_loader = DataLoader(val, batch_size = 64, shuffle = True)



In [ ]:
import torch.utils


class ImageTransferLearning(pl.LightningModule):
    def __init__(self):
        super().__init__()

        self.accuracy = torchmetrics.Accuracy(task="binary")
        backbone = models.resnet50(pretrained = True)
        num_features = backbone.fc.in_features    #capturing the number of features in the last layer of the backbone in this case resnet50 the last layer is a linear layer with 2048 features
        layers = list(backbone.children())[:-1]   # capturing all the layers of the backbone except the last layer that means all convolutional layers

        self.feature_extractor = nn.Sequential(*layers)  # creating a sequential model with all the layers of the backbone except the last layer
        num_target_classes = 2
        self.classifier = nn.Linear(num_features, num_target_classes)           # creating a linear layer with 2048 input features and 2 output features

    def forward(self, x):

            self.feature_extractor.eval()              # setting the feature extractor to evaluation mode that means the weights of the feature extractor will not be updated
            with torch.no_grad():
                representations = self.feature_extractor(x).flatten(1)          # that is a representation of the input image that is passed through the feature extractor that is frozen layer and not updated
            x = self.classifier(representations)                # passing the output of the feature extractor through the our classifier that make 2048 input features to 2 output features
            return F.softmax(x, dim =1)
        
    def train_dataloader(self):
            return torch.utils.data.DataLoader(train, batch_size = 64, shuffle = True)
        
    def val_dataloader(self):
            return torch.utils.data.DataLoader(val, batch_size = 64, shuffle = True)


    def training_step(self, batch, batch_idx):
            data, label = batch
            output = self.forward(data)
            loss = self.cross_entropy_loss(output, label)
            preds = torch.argmax(output, dim=1)
            self.log('train_loss', loss)
            self.log('train_acc', self.accuracy(preds, label))
            return {'loss': loss, 'lof': loss}
        
    def on_train_epoch_end(self):
            self.log('train_acc_epoc' , self.accuracy.compute()) 
        
    def validation_step(self, batch , batch_idx):
            val_data, val_label = batch
            output = self.forward(val_data)
            loss = self.cross_entropy_loss(output, val_label)
            preds = torch.argmax(output, dim=1)
            self.log('val_loss', loss)
            self.log('val_acc', self.accuracy(preds, val_label))

    def on_validation_epoch_end(self):
            self.log('val_acc_epoc' , self.accuracy.compute())    
            

            
            
    def cross_entropy_loss(self, logits, labels):
            return F.nll_loss(logits,labels)
        
    def configure_optimizers(self):
            optimizer = optim.Adam(self.parameters(), lr = 0.001)
            return optimizer

In [ ]:
model =ImageTransferLearning()

trainer = pl.Trainer(
    accelerator='gpu',
    max_epochs=10)

trainer.fit(model)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type           | Params
-----------------------------------------------------
0 | accuracy          | BinaryAccuracy | 0     
1 | feature_extractor | Sequential     | 23.5 M
2 | classifier        | Linear         | 4.1 K 
-----------------------------------------------------
23.5 M    Trainable params
0         Non-trainable params
23.5 M    Total params
94.049    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=2` reached.


In [ ]:
# Start tensorboard.
%load_ext tensorboard
%tensorboard --logdir lightning_logs/